In [29]:
import pandas as pd
import numpy as np
import selenium
import sqlalchemy
from urllib import parse

In [27]:
import requests
from bs4 import BeautifulSoup as BS
import os
import json

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}

def ev_cs_get_by_metro_code(result_file_name:str, API_KEY:str, metro_code:str, city_code=None)->bool:
    ev_api_url = "https://bigdata.kepco.co.kr/openapi/v1/EVcharge.do?{}{}apiKey={}&returnType=json"
    metroCd = f"metroCd={metro_code}&"
    cityCd = "" if city_code==None else f"cityCd={city_code}&"
    try:
        req = requests.get(ev_api_url.format(metroCd, cityCd, API_KEY))
        soup = BS(req.text)
        data = json.loads(soup.text)
        result_path = "./results"
        if not os.path.isdir(result_path):
            os.mkdir(result_path)
        with open(f"{result_path}/{result_file_name}", "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)
            
    except Exception as e:
        print(f"[debug] > exception : {e}")
        return False
    return True
    

In [28]:
metro_codes =  {
    "서울특별시" : "11",
    "부산광역시" : "21",
    "대구광역시" : "22",
    "인천광역시" : "23",
    "광주광역시" : "24",
    "대전광역시" : "25",
    "울산광역시" : "26",
    "세종특별자치시" : "29",
    "경기도" : "31",
    "강원도" : "32",
    "충청북도" : "33",
    "충청남도" : "34",
    "전라북도" : "35",
    "전라남도" : "36",
    "경상북도" : "37",
    "경상남도" : "38",
    "제주특별자치도" : "39"
}
with open("api_key.json", "r") as f:
    API_KEY = json.load(f)['API_KEY']

result_file_name_template = "ev_cs_{}.json"

for key in metro_codes.keys():
    assert ev_cs_get_by_metro_code(result_file_name_template.format(key), API_KEY, metro_code=metro_codes[key]), print(f"[debug] > {key} extraction failed")
    print(f"[debug] > {key} extraction success")

[debug] > 서울특별시 extraction success
[debug] > 부산광역시 extraction success
[debug] > 대구광역시 extraction success
[debug] > 인천광역시 extraction success
[debug] > 광주광역시 extraction success
[debug] > 대전광역시 extraction success
[debug] > 울산광역시 extraction success
[debug] > 세종특별자치시 extraction success
[debug] > 경기도 extraction success
[debug] > 강원도 extraction success
[debug] > 충청북도 extraction success
[debug] > 충청남도 extraction success
[debug] > 전라북도 extraction success
[debug] > 전라남도 extraction success
[debug] > 경상북도 extraction success
[debug] > 경상남도 extraction success
[debug] > 제주특별자치도 extraction success


In [32]:
with open("db_info.json", "r") as f:
    db_info = json.load(f)

user = db_info['user']
password = db_info['password']
host = db_info['host']
port = db_info['port']
database = db_info['database']
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")